In [18]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
n_fft = 512

In [12]:
file_path = 'singer_diarization_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Assuming 'time_start' and 'end_time' are in milliseconds
time_step = n_fft * 1000/22050  # Define your desired time step in milliseconds

# Calculate the total time span
total_time = df['end_time'].max()

# Create an array with zeros initially
label_array = np.zeros(int(total_time / time_step) + 1)

# Iterate through rows and update the array based on the label
for index, row in df.iterrows():
    start_idx = int(row['start_time'] / time_step)
    end_idx = int(row['end_time'] / time_step)

    if row['label'] in ['singer1', 'singer2']:
        label_array[start_idx:end_idx+1] = 1

# Cast the array to integer type
label_array = label_array.astype(int)

print(label_array)

[1 1 1 ... 1 1 1]


In [13]:
# Create a DataFrame with the time_array
result_df = pd.DataFrame({'time': np.arange(0, total_time + 1, time_step), 'value': label_array})

# Write the DataFrame to a new CSV file
result_df.to_csv('result.csv', index=False)

In [14]:
#Ckeck the no. of -ve and +ve values to check class imbalance
zero = 0
one = 0

for i in label_array:
    if i==0:
        zero+=1
    elif i==1:
        one+=1

print(zero, ", ", one)

#Find class imbalance
print(zero/one*100, "% Negative Values")

29534 ,  49208
60.018696146967976 % Negative Values


In [16]:
input = "singer_diarization.mp3"
#Load audio
signal, sr = librosa.load(input)

#Calculate MFCCs, delta MFCCs and delta delta MFCCs and concatenate them to get feature vector
mfcc = librosa.feature.mfcc(y=signal, n_mfcc=13, n_fft=n_fft, sr=sr)
del_mfcc = librosa.feature.delta(mfcc)
del2_mfcc = librosa.feature.delta(mfcc, order=2)
mfcc_features = np.concatenate((mfcc, del_mfcc, del2_mfcc))

mfcc_features = np.transpose(mfcc_features)

#Genrate train and test data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(mfcc_features, label_array, test_size=0.2, random_state=42)

In [17]:
import tensorflow as tf
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import ModelCheckpoint

#Define the model architecture
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(78, activation='relu', input_shape = (39,)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(156, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(25, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(5, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# Create an instance of the F1Score metric.
f1_score = F1Score(num_classes=2, average='micro')

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy', f1_score])

c:\Users\chinm\anaconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\chinm\anaconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [20]:
print(x_train.shape)
print(y_train.shape)

(62993, 39)
(62993,)


In [24]:
i=1

In [25]:

filepath="./ckpt" + str(i) + "/wght-imprv-{epoch:02d}-{accuracy:.2f}-{f1_score:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='f1_score', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(x_train, y_train, epochs=100, callbacks=callbacks_list)

Epoch 1/100
1964/1969 [============================>.] - ETA: 0s - loss: 0.0048 - accuracy: 0.9992 - f1_score: 0.7703
Epoch 1: f1_score improved from -inf to 0.77033, saving model to ./ckpt1\wght-imprv-01-1.00-0.77.hdf5
1969/1969 [==============================] - 14s 7ms/step - loss: 0.0048 - accuracy: 0.9992 - f1_score: 0.7703
Epoch 2/100
1964/1969 [============================>.] - ETA: 0s - loss: 0.0059 - accuracy: 0.9990 - f1_score: 0.7704
Epoch 2: f1_score did not improve from 0.77033
1969/1969 [==============================] - 13s 7ms/step - loss: 0.0058 - accuracy: 0.9990 - f1_score: 0.7703
Epoch 3/100
1966/1969 [============================>.] - ETA: 0s - loss: 0.0059 - accuracy: 0.9991 - f1_score: 0.7704
Epoch 3: f1_score did not improve from 0.77033
1969/1969 [==============================] - 12s 6ms/step - loss: 0.0059 - accuracy: 0.9991 - f1_score: 0.7703
Epoch 4/100
1963/1969 [============================>.] - ETA: 0s - loss: 0.0050 - accuracy: 0.9991 - f1_score: 0.7703

In [26]:
model.evaluate(x_test,  y_test, verbose=2)

493/493 - 1s - loss: 0.0361 - accuracy: 0.9999 - f1_score: 0.7645 - 1s/epoch - 3ms/step


[0.03609231859445572, 0.9998729825019836, 0.7645420432090759]

In [27]:
model.save_weights('./ckpt1/100.hdf5')

In [29]:
for batch_size in [5,25,125,625,3125,15625,62993]:
    #Define the model architecture
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(78, activation='relu', input_shape = (39,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(156, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

    # Create an instance of the F1Score metric.
    f1_score = F1Score(num_classes=2, average='micro')

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy', f1_score])

    filepath="./batch_size-" + str(batch_size) + "/wght-imprv-{epoch:02d}-{accuracy:.2f}-{f1_score:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x_train, y_train, epochs=50, batch_size=batch_size, callbacks=callbacks_list)
    model.save_weights("./batch_size-" + str(batch_size) + "/50.hdf5")

Epoch 1/50
12593/12599 [============================>.] - ETA: 0s - loss: 0.1022 - accuracy: 0.9501 - f1_score: 0.8008
Epoch 1: accuracy improved from -inf to 0.95007, saving model to ./batch_size-5\wght-imprv-01-0.95-0.80.hdf5
12599/12599 [==============================] - 85s 7ms/step - loss: 0.1022 - accuracy: 0.9501 - f1_score: 0.8008
Epoch 2/50
12597/12599 [============================>.] - ETA: 0s - loss: 0.0333 - accuracy: 0.9913 - f1_score: 0.8977
Epoch 2: accuracy improved from 0.95007 to 0.99128, saving model to ./batch_size-5\wght-imprv-02-0.99-0.90.hdf5
12599/12599 [==============================] - 85s 7ms/step - loss: 0.0333 - accuracy: 0.9913 - f1_score: 0.8977
Epoch 3/50
12594/12599 [============================>.] - ETA: 0s - loss: 0.0259 - accuracy: 0.9932 - f1_score: 0.9058
Epoch 3: accuracy improved from 0.99128 to 0.99322, saving model to ./batch_size-5\wght-imprv-03-0.99-0.91.hdf5
12599/12599 [==============================] - 82s 6ms/step - loss: 0.0259 - accurac

In [30]:
model.load_weights("./batch_size-25/50.hdf5")
model.evaluate(x_test,  y_test, verbose=2)

493/493 - 2s - loss: 0.0025 - accuracy: 0.9996 - f1_score: 0.9997 - 2s/epoch - 3ms/step


[0.0024945661425590515, 0.9996190071105957, 0.9996922612190247]

In [31]:
model.load_weights("./batch_size-5/50.hdf5")
model.evaluate(x_test,  y_test, verbose=2)

493/493 - 2s - loss: 0.0102 - accuracy: 0.9996 - f1_score: 0.9956 - 2s/epoch - 3ms/step


[0.010198471136391163, 0.9995555281639099, 0.995607316493988]